In [38]:
import gspread as gspread
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib

In [39]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    EnsureType,
    Invertd,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import re



from monai.transforms import (
    LoadImage, LoadImaged, EnsureChannelFirstd,
    Resized, EnsureTyped, Compose
)
from monai.config import print_config

print_config()

MONAI version: 0.8.0
Numpy version: 1.21.5
Pytorch version: 1.10.1+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.7
Nibabel version: 3.2.1
scikit-image version: 0.19.1
Pillow version: 8.4.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 4.2.0
TorchVision version: 0.11.2+cu113
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [41]:
set_determinism(seed=0)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear", "nearest")),#standardizing the size of image ...
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)


        # ScaleIntensityRanged(
        #     keys=["image"], a_min=-57, a_max=164,
        #     b_min=0.0, b_max=1.0, clip=True,
        # ),

In [42]:
data_dir = 'C:\\Users\\1\\Downloads\\volumes 0-49'

train_images = sorted(
    glob.glob(os.path.join(data_dir, "volumes 0-49", "*.nii.gz")))

train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labels", "*.nii.gz")))

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
data_dicts
#train_files, val_files = data_dicts[:-9], data_dicts[-9:]

[{'image': 'C:\\Users\\1\\Downloads\\volumes 0-49\\volumes 0-49\\volume-19.nii.gz',
  'label': 'C:\\Users\\1\\Downloads\\volumes 0-49\\labels\\labels-19.nii.gz'},
 {'image': 'C:\\Users\\1\\Downloads\\volumes 0-49\\volumes 0-49\\volume-20.nii.gz',
  'label': 'C:\\Users\\1\\Downloads\\volumes 0-49\\labels\\labels-20.nii.gz'},
 {'image': 'C:\\Users\\1\\Downloads\\volumes 0-49\\volumes 0-49\\volume-21.nii.gz',
  'label': 'C:\\Users\\1\\Downloads\\volumes 0-49\\labels\\labels-21.nii.gz'}]

In [43]:
import h5py
f = h5py.File("D:\\dataSets\\forMainHDF5\\CtOrg.hdf5", "w")

In [46]:
check_ds = Dataset(data=data_dicts, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
#looking for biggest image for metrics testing
biggestDat = 0
index = 0
maxxx = 0
for dat in check_loader:
    index+=1
    #### getting number describing patient
    string = str(dat['image_meta_dict']['filename_or_obj'])
    numb = re.search('.nii', string).span()[0] ## this does what you wanted
    describingNumber = string[(numb-2):(numb)]
    #group collecting
    grp = f.create_group(describingNumber)
    imageDst = grp.create_dataset('image', data=  dat['image'].cpu().detach().numpy())
    labelDst= grp.create_dataset('label', data=  dat['label'].cpu().detach().numpy())
    #setting attributes needed for later identification
    imageDst.attrs['studyType'] = 'CT';
    labelDst.attrs['studyType'] = 'multiLabel';

    print(describingNumber)
    # if index==14:
    #     biggestDat = dat
    #     break

19
20
21


In [ ]:
['C:\\Users\\1\\Downloads\\volumes 0-49\\volumes 0-49\\volume-19.nii.gz']

Closing  connection to hdf5

In [47]:
f.close()